In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,clone_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

from utils import *

Using TensorFlow backend.


In [2]:
df = functions.parse_file(r"raw_data/EmoContext/train2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [3]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [4]:
NR_WORDS = 8000
MAX_PROP_LENGTH = 128
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [5]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train = []

for idx,row in df.iterrows():
    Y_train.append(one_hot_vector(row['label']))

Y_train = np.array(Y_train)

In [6]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,angry
1,1,Mixed things such as??,the things you do.,Have you seen minions??,others
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,happy
3,3,Woah bring me some,left it there oops,Brb,others
4,4,it is thooooo,I said soon master.,he is pressuring me,others


In [7]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [8]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [9]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test = []

for idx,row in df.iterrows():
    Y_test.append(one_hot_vector(row['label']))

Y_test = np.array(Y_test)

In [10]:
final_loaded_models = load_models()
merged_layer, input_layers = create_final_layers(final_loaded_models)
merged_layer = Dense(8)(merged_layer)
merged_layer = Dropout(0.5)(merged_layer)
merged_layer = Dense(8)(merged_layer)
merged_layer = Dropout(0.5)(merged_layer)
merged_layer = Dense(8)(merged_layer)
merged_layer = Dropout(0.5)(merged_layer)
merged_layer = Dense(4, activation='softmax')(merged_layer)
merged_model = Model(inputs=input_layers, output=merged_layer,name="merged_model")
merged_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy',f1])

Loaded model models\angry_sequential_1_model.json from disk
Loaded model models\happy_sequential_1_model.json from disk
Loaded model models\others_model_model.json from disk
Loaded model models\sad_sequential_1_model.json from disk
Loaded model models\sin_sequential_1_model.json from disk


C:\Python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [11]:
merged_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
angry_boosting_embeding_input ( (None, 200)          0                                            
__________________________________________________________________________________________________
happy_boosting_embeding_input ( (None, 200)          0                                            
__________________________________________________________________________________________________
others_boosting_embeding_input  (None, 200)          0                                            
__________________________________________________________________________________________________
sad_boosting_embeding_input (In (None, 200)          0                                            
__________________________________________________________________________________________________
sin_model_

In [12]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [13]:
batch_size = 64

Y_train = np.array(Y_train)
total = len(Y_train)
history = merged_model.fit([X_train,X_train,X_train,X_train,X_train], Y_train,
                    validation_data=([X_test,X_test,X_test,X_test,X_test], Y_test),
                    epochs=10, verbose=1, batch_size=batch_size,class_weight={
                        0: total / len(np.where(Y_train[:,0]==1.0)[0]),
                        1: total / len(np.where(Y_train[:,1]==1.0)[0]),
                        2: total / len(np.where(Y_train[:,2]==1.0)[0]),
                        3: total / len(np.where(Y_train[:,3]==1.0)[0]),
                    },callbacks=[mdcheck],
                          shuffle=True)

Train on 30160 samples, validate on 2755 samples
Epoch 1/10
30160/30160 [==============================] - 70s 2ms/step - loss: 3.9035 - acc: 0.5672 - f1: 0.4705 - val_loss: 1.8513 - val_acc: 0.4868 - val_f1: 0.2175
Epoch 2/10
30160/30160 [==============================] - 70s 2ms/step - loss: 2.3993 - acc: 0.7769 - f1: 0.7257 - val_loss: 2.6100 - val_acc: 0.4947 - val_f1: 0.2282
Epoch 3/10
30160/30160 [==============================] - 72s 2ms/step - loss: 2.1117 - acc: 0.8180 - f1: 0.7722 - val_loss: 2.7948 - val_acc: 0.4991 - val_f1: 0.2330
Epoch 4/10
30160/30160 [==============================] - 71s 2ms/step - loss: 1.9625 - acc: 0.8393 - f1: 0.7974 - val_loss: 3.1064 - val_acc: 0.4955 - val_f1: 0.2316
Epoch 5/10
30160/30160 [==============================] - 73s 2ms/step - loss: 1.9152 - acc: 0.8529 - f1: 0.8146 - val_loss: 3.1765 - val_acc: 0.5013 - val_f1: 0.2327
Epoch 6/10
30160/30160 [==============================] - 71s 2ms/step - loss: 1.8528 - acc: 0.8632 - f1: 0.8293 - v

In [ ]:
save_model(merged_model)

In [14]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df_test.head()

,id,turn1,turn2,turn3
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF
1,1,Mixed things such as??,the things you do.,Have you seen minions??
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry
3,3,Woah bring me some,left it there oops,Brb
4,4,it is thooooo,I said soon master.,he is pressuring me


In [15]:
text_data = []

for idx,row in df_test.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

res = model.predict(X_test, batch_size=64, verbose=1)
res[3]

NameError: name 'model' is not defined

In [ ]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [ ]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(50)
df_test.to_csv("combined_models_submission.txt",index=False , sep="\t")